# 플레이어 이탈 데이터셋을 위한 특성 공학

이 노트북은 player_churn.csv 데이터셋에 대한 특성 공학을 수행하여 추가 분석을 위해 특정 열을 선택합니다. 특성 공학은 머신 러닝 파이프라인에서 가장 관련성 높은 특성을 선택하여 모델 성능을 향상시키는 중요한 단계입니다.

이 노트북에서는 다음 사항에 중점을 둘 것입니다:
1. 플레이어 이탈 데이터셋 로드
2. 도메인 지식을 기반으로 가장 중요한 특성 선택
3. 선택된 특성 탐색
4. 모델 훈련을 위한 처리된 데이터셋 저장

In [ ]:
%pip install scikit-learn "pandas>=2.0.0" s3fs==0.4.2 sagemaker xgboost mlflow==2.13.2 sagemaker-mlflow==0.1.0 seaborn

## 환경 설정

먼저, 데이터 조작, 분석 및 시각화를 위한 필요한 라이브러리를 가져오겠습니다.

In [2]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.utils import resample
import boto3
import time
from datetime import datetime

# Set display options
pd.set_option('display.max_columns', None)

## 데이터셋 로드

플레이어 행동 데이터와 이탈 정보가 포함된 player_churn.csv 파일을 로드하겠습니다. 이 데이터셋에는 플레이어 세션, 참여 패턴 및 이탈 여부(게임 중단)에 관한 다양한 지표가 포함되어 있습니다.

In [ ]:
# Load the dataset
df = pd.read_csv('data/player_churn.csv')

# Display basic information
print(f"Dataset shape: {df.shape}")
df.head()

## 특성 공학

이 단계에서는 이탈 예측 모델에 가장 관련성 있는 특성만 선택하겠습니다. 이러한 특성들은 도메인 전문 지식이나 이전 분석을 통해 플레이어 이탈을 예측하는 데 가장 효과적인 것으로 확인되었을 가능성이 높습니다.

선택된 특성에는 다음이 포함됩니다:
- 플레이어 식별 특성(`player_id`, `cohort_id`, `player_type`)
- 시간적 특성(`cohort_day_of_week`)
- 참여 지표(`player_lifetime`, `session_count`)
- 세션 타이밍 패턴(다양한 시간대 지표)
- 타겟 변수(`player_churn`)

이러한 특정 특성에 집중함으로써 더 효율적이고 해석 가능한 모델을 구축할 수 있습니다.

In [ ]:
# Define columns to keep
cols = ['cohort_day_of_week',
       'begin_session_time_of_day_std_last_week_1',
       'player_lifetime',
       'begin_session_time_of_day_mean_last_day_1',
       'end_session_time_of_day_mean_last_week_1',
       'begin_session_time_of_day_mean_last_week_1',
       'cohort_id',
       'player_type',
       'begin_session_time_of_day_std_last_day_1',
       'end_session_time_of_day_mean_last_day_1',
       'begin_session_time_of_day_mean_last_day_2',
       'end_session_time_of_day_std_last_week_1',
       'end_session_time_of_day_std_last_day_1',
       'session_count',
       'end_session_time_of_day_mean_last_day_3',
       'begin_session_time_of_day_mean_last_day_3',
       'end_session_time_of_day_mean_last_day_2',
       'player_churn',
       'player_id']

# Check if all columns exist in the dataset
missing_cols = [col for col in cols if col not in df.columns]
if missing_cols:
    print(f"Warning: The following columns are not in the dataset: {missing_cols}")
    # Keep only columns that exist in the dataset
    cols = [col for col in cols if col in df.columns]

# Select only the specified columns
df_selected = df[cols]

# Display the resulting dataframe
print(f"Selected dataset shape: {df_selected.shape}")
df_selected.head()

## 결측값 처리

특정 시간대 표준 편차 특성의 경우, 결측값을 0으로 채우겠습니다. 이는 결측값이 세션 시간의 변동이 없음을 나타낼 가능성이 높기 때문에(예: 세션이 하나뿐이거나 일관된 세션 시간) 이러한 특성에 적합합니다.

In [ ]:
# List of columns to fill with 0
fill_zero_cols = [
    'begin_session_time_of_day_std_last_week_1',
    'begin_session_time_of_day_std_last_day_1',
    'end_session_time_of_day_std_last_week_1',
    'end_session_time_of_day_std_last_day_1'
]

# Fill missing values with 0 for specified columns
for col in fill_zero_cols:
    if col in df_selected.columns:
        # Count missing values before filling
        missing_count = df_selected[col].isnull().sum()
        if missing_count > 0:
            print(f"Filling {missing_count} missing values with 0 in column: {col}")
            df_selected.loc[:, col] = df_selected[col].fillna(0)

# Verify the missing values were filled
missing_after = {col: df_selected[col].isnull().sum() for col in fill_zero_cols if col in df_selected.columns}
print("\nRemaining missing values after filling:")
for col, count in missing_after.items():
    print(f"{col}: {count}")

## 원-핫 인코딩

범주형 변수에 원-핫 인코딩을 적용하여 머신 러닝 알고리즘에 제공할 수 있는 형식으로 변환하겠습니다. 이 과정은 원래 범주형 열의 각 카테고리에 대한 이진 열을 생성합니다.

In [ ]:
# Check unique values in categorical columns before encoding
if 'player_type' in df_selected.columns:
    print(f"Unique values in player_type: {df_selected['player_type'].nunique()}")
    print(df_selected['player_type'].value_counts())
    
if 'cohort_id' in df_selected.columns:
    print(f"\nUnique values in cohort_id: {df_selected['cohort_id'].nunique()}")
    print(df_selected['cohort_id'].value_counts().head())  # Show only top values if many

In [ ]:
# Apply one-hot encoding
# Create dummy variables for player_type and cohort_id
cols_to_encode = ['player_type', 'cohort_id']
encoded_cols = [col for col in cols_to_encode if col in df_selected.columns]

if encoded_cols:
    # Get one-hot encoding
    df_encoded = pd.get_dummies(df_selected, columns=encoded_cols, prefix=encoded_cols, dtype=int)
    
    # Display information about the encoded dataset
    print(f"Shape before encoding: {df_selected.shape}")
    print(f"Shape after encoding: {df_encoded.shape}")
    print(f"New columns added: {df_encoded.shape[1] - df_selected.shape[1]}")
    
    # Update our working dataframe
    df_selected = df_encoded
    
    # Show a sample of the encoded columns
    encoded_column_names = [col for col in df_selected.columns if any(col.startswith(prefix + '_') for prefix in encoded_cols)]
    print("\nSample of encoded columns:")
    print(encoded_column_names[:10])  # Show first 10 encoded columns
    
    # Display the first few rows of the encoded dataframe
    df_selected.head()

## 특성 선택 후 데이터 탐색

이제 특성을 선택하고, 결측값을 처리하고, 범주형 변수를 인코딩했으니 데이터셋을 탐색하여 작업할 데이터를 더 잘 이해해 보겠습니다.

### 결측값 분석

선택한 특성에 남아있는 결측값이 있는지 확인해 보겠습니다. 결측값은 모델 성능에 상당한 영향을 미칠 수 있으므로 적절하게 처리해야 합니다.

In [ ]:
# Check for missing values
missing_values = df_selected.isnull().sum()
print("Missing values per column:")
print(missing_values[missing_values > 0])

### 통계 요약

수치형 특성의 기본 통계를 살펴보고 분포, 범위 및 잠재적 이상치를 이해해 보겠습니다.

In [ ]:
# Basic statistics of numerical columns
df_selected.describe()

### 타겟 변수 분석

타겟 변수(player_churn)의 분포를 이해하는 것은 모델 개발에 중요합니다. 불균형한 분포는 모델 훈련 중에 특별한 처리 기법이 필요할 수 있습니다.

In [ ]:
# Distribution of target variable
if 'player_churn' in df_selected.columns:
    plt.figure(figsize=(8, 6))
    sns.countplot(x='player_churn', data=df_selected)
    plt.title('Distribution of Player Churn')
    plt.xlabel('Player Churn (0 = No, 1 = Yes)')
    plt.ylabel('Count')
    plt.show()
    
    # Calculate churn rate
    churn_rate = df_selected['player_churn'].mean() * 100
    print(f"Churn rate: {churn_rate:.2f}%")

## 데이터셋 균형 맞추기

모델 성능을 향상시키기 위해 랜덤 오버샘플링을 사용하여 데이터셋의 균형을 맞추겠습니다. 이 기법은 소수 클래스(이탈한 플레이어)의 샘플을 무작위로 복제하여 클래스 간 1:1 비율을 달성하는 것을 포함합니다.

In [ ]:
# Separate majority and minority classes
df_majority = df_selected[df_selected['player_churn'] == 0]
df_minority = df_selected[df_selected['player_churn'] == 1]

print(f"Before oversampling:\n"
      f"Number of non-churned players (majority): {len(df_majority)}\n"
      f"Number of churned players (minority): {len(df_minority)}")

# Oversample minority class
df_minority_oversampled = resample(df_minority, 
                                   replace=True,     # sample with replacement
                                   n_samples=len(df_majority),    # match majority class
                                   random_state=42)  # reproducible results

# Combine majority class with oversampled minority class
df_balanced = pd.concat([df_majority, df_minority_oversampled])

# Display new class distribution
print(f"\nAfter oversampling:\n"
      f"Number of non-churned players: {len(df_balanced[df_balanced['player_churn'] == 0])}\n"
      f"Number of churned players: {len(df_balanced[df_balanced['player_churn'] == 1])}")

# Shuffle the balanced dataset
df_balanced = df_balanced.sample(frac=1, random_state=42).reset_index(drop=True)

# Visualize the balanced distribution
plt.figure(figsize=(8, 6))
sns.countplot(x='player_churn', data=df_balanced)
plt.title('Distribution of Player Churn After Balancing')
plt.xlabel('Player Churn (0 = No, 1 = Yes)')
plt.ylabel('Count')
plt.show()

## 데이터 타입 변환

일부 머신 러닝 알고리즘은 특정 데이터 타입을 요구합니다. 여기서는 타겟 변수를 불리언에서 정수(long) 형식으로 변환하겠습니다.

In [ ]:
# Check current data type of player_churn
print(f"Current data type of player_churn: {df_balanced['player_churn'].dtype}")

# Convert player_churn from boolean to long (int64)
df_balanced['player_churn'] = df_balanced['player_churn'].astype('int64')

# Verify the conversion
print(f"New data type of player_churn: {df_balanced['player_churn'].dtype}")

# Display a sample of the data to confirm
print("\nSample values after conversion:")
print(df_balanced['player_churn'].head())

## SageMaker Feature Store에 저장

이제 처리된 데이터셋을 머신 러닝 워크플로우에서 사용하기 위해 Amazon SageMaker Feature Store에 저장하겠습니다. Feature Store는 특성을 위한 중앙 저장소를 제공하여 팀과 프로젝트 간에 특성을 더 쉽게 공유하고 재사용할 수 있게 합니다.

### SageMaker Feature Store란 무엇인가?

Amazon SageMaker Feature Store는 특성을 저장하고 액세스할 수 있는 목적별 저장소로, 팀 간에 특성을 더 쉽게 이름 지정, 구성 및 재사용할 수 있습니다. 주요 이점은 다음과 같습니다:

- **특성 재사용**: 특성을 한 번 저장하고 여러 모델에서 재사용
- **일관성**: 훈련과 추론 간에 일관된 특성 변환 보장
- **발견 가능성**: 조직 전체에서 특성을 발견하고 공유 가능
- **실시간 액세스**: 온라인 추론을 위한 낮은 지연 시간으로 특성 액세스
- **과거 액세스**: 훈련 및 백테스팅을 위한 특정 시점의 특성 값 검색

In [ ]:
# Import SageMaker Feature Store modules
import sagemaker
from sagemaker.session import Session
from sagemaker.feature_store.feature_group import FeatureGroup
from sagemaker.feature_store.feature_definition import FeatureDefinition, FeatureTypeEnum

# Initialize SageMaker session
session = sagemaker.Session()
region = session.boto_region_name
s3_bucket_name = session.default_bucket()
prefix = "player-churn-feature-store"
role = sagemaker.get_execution_role()

print(f"SageMaker session initialized in region: {region}")
print(f"Using S3 bucket: {s3_bucket_name}")

### Feature Store를 위한 데이터 준비

SageMaker Feature Store는 두 개의 특수 열이 필요합니다:
1. 각 레코드를 고유하게 식별하는 **레코드 식별자** 열(여기서는 `player_id` 사용)
2. 특성 값이 생성된 시점을 나타내는 **이벤트 시간** 열

데이터셋에 이벤트 시간 열을 추가하겠습니다.

In [13]:
# Table is available as variable `df`
from datetime import datetime, timezone

def generate_event_timestamp():
    # naive datetime representing local time
    naive_dt = datetime.now()
    # take timezone into account
    aware_dt = naive_dt.astimezone()
    # time in UTC
    utc_dt = aware_dt.astimezone(timezone.utc)
    # transform to ISO-8601 format
    event_time = utc_dt.isoformat(timespec='milliseconds')
    event_time = event_time.replace('+00:00', 'Z')
    return event_time



In [ ]:
# Create a new column 'EventTime' with the current timestamp. This will be used as the event time for the feature store.
dt = generate_event_timestamp()
df_balanced['event_time'] = dt

# Define feature group name
feature_group_name = "player-churn-features-" + datetime.now().strftime("%Y-%m-%d-%H-%M-%S")

# Create Feature Group
player_churn_feature_group = FeatureGroup(name=feature_group_name, sagemaker_session=session)

# Load the data into Feature Store
player_churn_feature_group.load_feature_definitions(data_frame=df_balanced)
print(f"Feature group name: {feature_group_name}")
print("Feature definitions loaded from dataframe")

### 특성 그룹 생성

이제 SageMaker Feature Store에 특성 그룹을 생성하겠습니다. 다음과 같이 구성하겠습니다:
- 오프라인 저장을 위한 S3 위치
- 레코드 식별자 열(`player_id`)
- 이벤트 시간 열(`EventTime`)
- 낮은 지연 시간 액세스를 위한 온라인 스토어 활성화

In [ ]:
# Create the feature group
player_churn_feature_group.create(
    s3_uri=f"s3://{s3_bucket_name}/{prefix}",
    record_identifier_name="player_id",
    event_time_feature_name="event_time",
    role_arn=role,
    enable_online_store=True
)

# Wait for feature group creation to complete
# player_churn_feature_group.wait()
# print(f"Feature group {feature_group_name} created successfully")

In [ ]:
import time
def wait_for_feature_group_creation_complete(feature_group):
    """Helper function to wait for the completions of creating a feature group"""
    response = feature_group.describe()
    status = response.get("FeatureGroupStatus")
    while status == "Creating":
        print("Waiting for Feature Group Creation")
        time.sleep(5)
        response = feature_group.describe()
        status = response.get("FeatureGroupStatus")

    if status != "Created":
        print(f"Failed to create feature group, response: {response}")
        failureReason = response.get("FailureReason", "")
        raise SystemExit(
            f"Failed to create feature group {feature_group.name}, status: {status}, reason: {failureReason}"
        )
    print(f"FeatureGroup {feature_group.name} successfully created.")

wait_for_feature_group_creation_complete(feature_group=player_churn_feature_group)

### Feature Store에 데이터 수집

마지막으로, 처리된 데이터를 특성 그룹에 수집하겠습니다. 이렇게 하면 다음과 같은 용도로 특성을 사용할 수 있게 됩니다:
- 새 모델 훈련
- 실시간 추론
- 특성 탐색 및 분석
- 다른 팀과의 공유

In [ ]:
# Ingest data into the feature group
player_churn_feature_group.ingest(data_frame=df_balanced, max_workers=3, wait=True)
print(f"Ingested {len(df_balanced)} records into feature group {feature_group_name}")

# Describe the feature group to verify
feature_group_details = player_churn_feature_group.describe()
print(f"\nFeature Group Details:\n{feature_group_details}")

## 결론

이 노트북에서는 플레이어 이탈 데이터셋에 대한 포괄적인 특성 공학을 수행했습니다:

1. 가장 관련성 있는 특성 선택
2. 결측값 처리
3. 범주형 변수에 원-핫 인코딩 적용
4. 오버샘플링을 사용하여 데이터셋 균형 맞추기
5. ML 알고리즘과의 호환성을 위한 데이터 타입 변환
6. ML 워크플로우에서 재사용하기 위해 SageMaker Feature Store에 특성 저장

처리된 데이터는 이제 모델 훈련 및 평가를 위한 준비가 완료되었습니다.